In [1]:
import logging

# Set up logging
logging.basicConfig(filename='debug.log', level=logging.DEBUG)

# Then in your code...
def some_function():
    logging.info('Starting function...')
    # ...code...
    logging.info('Finished function.')

In [2]:
import numpy as np
np.__version__

'2.0.2'

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.__version__

2025-08-23 13:53:26.332440: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-23 13:53:26.358117: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-23 13:53:26.401401: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-23 13:53:26.424102: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755971606.449113 1891241 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755971606.45

Num GPUs Available:  0


2025-08-23 13:53:31.709889: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


'2.19.0'

In [4]:
from importlib import reload
import sys
import os
os.chdir('/projects/sccn/weili_A1/liacz/Documents/Bilingualism_CVAE')
#set up path to import helper_funcs
sys.path.append('/projects/sccn/weili_A1/liacz/Documents/Bilingualism_CVAE/misc')
from helper_funcs import *
from make_models2 import *

# Make tqdm work for notebooks
from functools import partial #already installed in conda environment
from tqdm import tqdm
tqdm = partial(tqdm, position=0, leave=True)

from matplotlib import pyplot as plt
# import seaborn as sns

import numpy as np
import pandas as pd

import pickle
# from sklearn.metrics import silhouette_score
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

# Load data

In [5]:
import numpy as np
import os
BRAIN_data = np.load("data/array_brains/arr_combine/oct24/BRAIN_data.npy")
DX_subs = np.load("data/array_brains/arr_combine/oct24/DX_subs.npy")
TD_subs = np.load("data/array_brains/arr_combine/oct24/TD_subs.npy")

In [ ]:
# #BRAIN_data = np.array(BRAIN_data)
# nsubs = BRAIN_data.shape[0]
# print([arr.shape for arr in [BRAIN_ds, BRAIN_subs, BRAIN_data]])
# print(nsubs)
# print((BRAIN_data.min(),BRAIN_data.max()))

In [ ]:
# df = pd.read_csv('/mmfs1/data/liacz/Documents/Bilingualism_CVAE/data/legend.csv')
# BI_subs = df.loc[df['bilingualism'].values == 1.0, 'ds_par_id']
# MO_subs = df.loc[df['bilingualism'].values == 0.0, 'ds_par_id']
# BRAIN_ds_subs = [BRAIN_ds[i]+'_'+BRAIN_subs[i] for i in range(0,len(BRAIN_subs))]
# BI_subs = [sub in BI_subs.to_list() for sub in BRAIN_ds_subs]
# MO_subs = [sub in MO_subs.to_list() for sub in BRAIN_ds_subs]

In [ ]:
# print(BI_subs.count(True))
# print(MO_subs.count(True))

In [6]:
# TD_subs = BRAIN_data[MO_subs,:,:,:] # Data of Monolinguals 
# DX_subs = BRAIN_data[BI_subs,:,:,:] # Data of Bilinguals
print(TD_subs.shape)
print(DX_subs.shape)

(1644, 64, 64, 64)
(853, 64, 64, 64)


In [7]:
!nvidia-smi

Mon Aug 18 20:43:12 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.133.20             Driver Version: 570.133.20     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          Off |   00000000:17:00.0 Off |                    0 |
| N/A   32C    P0             45W /  300W |       4MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

# train VAE

In [35]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Conv3D, Conv3DTranspose, Flatten, Dense, Reshape, Lambda
)
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model


# 1) The reparameterization trick
def sampling(args):
    z_mean, z_log_var = args
    batch   = K.shape(z_mean)[0]
    dim     = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


# 2) Build encoder + decoder (no add_loss, no Functional‐API VAE)
def build_encoder_decoder(input_shape, latent_dim,
                          nlayers=2, base_filters=32,
                          kernel_size=3, intermediate_dim=128,
                          use_bias=True):
    # -- Encoder
    inputs = Input(shape=input_shape, name="encoder_input")
    x = inputs
    filters = base_filters
    for _ in range(nlayers):
        filters *= 2
        x = Conv3D(filters=filters,
                   kernel_size=kernel_size,
                   strides=2,
                   padding="same",
                   activation="relu",
                   use_bias=use_bias)(x)
    shape_info = K.int_shape(x)  # needed for decoder reshape

    x = Flatten()(x)
    x = Dense(intermediate_dim, activation="relu", use_bias=use_bias)(x)
    z_mean    = Dense(latent_dim, name="z_mean", use_bias=use_bias)(x)
    z_log_var = Dense(latent_dim, name="z_log_var", use_bias=use_bias)(x)
    z         = Lambda(sampling, name="z")([z_mean, z_log_var])

    encoder = Model(inputs, [z_mean, z_log_var, z], name="encoder")

    # -- Decoder
    latent_inputs = Input(shape=(latent_dim,), name="z_sampling")
    x = Dense(intermediate_dim, activation="relu", use_bias=use_bias)(latent_inputs)
    total_dims = shape_info[1] * shape_info[2] * shape_info[3] * shape_info[4]
    x = Dense(total_dims, activation="relu", use_bias=use_bias)(x)
    x = Reshape((shape_info[1],
                 shape_info[2],
                 shape_info[3],
                 shape_info[4]))(x)

    filters = shape_info[4]
    for _ in range(nlayers):
        x = Conv3DTranspose(filters=filters,
                            kernel_size=kernel_size,
                            strides=2,
                            padding="same",
                            activation="relu",
                            use_bias=use_bias)(x)
        filters //= 2

    outputs = Conv3DTranspose(filters=input_shape[3],
                              kernel_size=kernel_size,
                              padding="same",
                              activation="sigmoid",
                              name="decoder_output",
                              use_bias=use_bias)(x)

    decoder = Model(latent_inputs, outputs, name="decoder")
    return encoder, decoder


# 3) Optional disentanglement helper
def compute_tc_and_disc(z, gamma):
    # batch‐split
    z_top, z_bottom = tf.split(z, 2, axis=0)
    # latent‐split
    z1_a, z1_b = tf.split(z_top,    2, axis=1)
    z2_a, z2_b = tf.split(z_bottom, 2, axis=1)

    # build positive / negative pairs
    q     = tf.concat([tf.concat([z1_b, z1_b], axis=1),
                       tf.concat([z2_b, z2_b], axis=1)],
                      axis=0)
    q_bar = tf.concat([tf.concat([z1_b, z2_b], axis=1),
                       tf.concat([z2_b, z1_b], axis=1)],
                      axis=0)

    discriminator = tf.keras.Sequential([
        Dense(1, activation="sigmoid")
    ], name="discriminator")

    eps = 1e-7
    q_score_raw     = discriminator(q)
    q_bar_score_raw = discriminator(q_bar)

    q_score     = tf.clip_by_value((q_score_raw     + 0.1) * 0.85, eps, 1-eps)
    q_bar_score = tf.clip_by_value((q_bar_score_raw + 0.1) * 0.85, eps, 1-eps)

    # total-correlation term
    tc_loss = tf.reduce_mean(tf.math.log(q_score / (1 - q_score)))
    # discriminator cross‐entropy term
    disc_loss = tf.reduce_mean(-tf.math.log(q_score)
                               - tf.math.log(1 - q_bar_score))

    return gamma * tc_loss, disc_loss


# 4) The subclassed VAE
class VAE(tf.keras.Model):
    def __init__(self,
                 encoder,
                 decoder,
                 image_size,
                 channels,
                 gamma=1.0,
                 disentangle=False,
                 **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.image_size = image_size
        self.channels   = channels
        self.gamma      = gamma
        self.disentangle = disentangle

        # Trackers
        self.total_loss_tracker = tf.keras.metrics.Mean(name="loss")
        self.recon_loss_tracker = tf.keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker    = tf.keras.metrics.Mean(name="kl_loss")
        if disentangle:
            self.tc_loss_tracker   = tf.keras.metrics.Mean(name="tc_loss")
            self.disc_loss_tracker = tf.keras.metrics.Mean(name="discriminator_loss")

    @property
    def metrics(self):
        m = [self.total_loss_tracker,
             self.recon_loss_tracker,
             self.kl_loss_tracker]
        if self.disentangle:
            m += [self.tc_loss_tracker, self.disc_loss_tracker]
        return m

    def train_step(self, data):
        x = tf.cast(data, tf.float32) # expecting shape (batch, X,Y,Z,channels)
        with tf.GradientTape() as tape:
            # 1) encode
            z_mean, z_log_var, z = self.encoder(x)
            # 2) decode
            x_recon = self.decoder(z)

            # 3) reconstruction loss (MSE × volume)
            vol_size = (self.image_size ** 3) * self.channels
            recon_loss = tf.reduce_mean(tf.square(x - x_recon)) * vol_size

            # 4) KL divergence
            kl = -0.5 * (1 + z_log_var
                         - tf.square(z_mean)
                         - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl, axis=1))

            total_loss = recon_loss + kl_loss

            # 5) (optional) disentanglement
            if self.disentangle:
                tc, disc = compute_tc_and_disc(z, gamma=self.gamma)
                total_loss += tc + disc

        # 6) backprop
        grads = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))

        # 7) update trackers
        self.total_loss_tracker.update_state(total_loss)
        self.recon_loss_tracker.update_state(recon_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        if self.disentangle:
            self.tc_loss_tracker.update_state(tc)
            self.disc_loss_tracker.update_state(disc)

        return {m.name: m.result() for m in self.metrics}

In [ ]:
if train_vae:
    
    latent_dim = 32
    batch_size = 16
    disentangle = False
    gamma = 100

    encoder, decoder, vae = get_MRI_VAE_3D(input_shape=(64,64,64,1), 
                                            latent_dim=32, 
                                            batch_size = batch_size, 
                                            disentangle=True,
                                            gamma=gamma,
                                            kernel_size = 3,
                                            filters = 48,
                                            intermediate_dim = 128,
                                            nlayers = 2,
                                            bias=True)


    loss = list()
    fn = '/mmfs1/data/liacz/Documents/Bilingualism_CVAE/data/tf_outputs/VAE/VAE_weights'
        
    print(fn)

In [36]:
input_shape = (64, 64, 64, 1)
latent_dim  = 32

encoder, decoder = build_encoder_decoder(
    input_shape=input_shape,
    latent_dim=latent_dim,
    nlayers=2,
    base_filters=32,
    intermediate_dim=128
)

# b) instantiate the subclassed VAE
vae = VAE(
    encoder=encoder,
    decoder=decoder,
    image_size=input_shape[0],
    channels=input_shape[3],
    gamma=100.0,
    disentangle=True,
    name="mri_vae_3d"
)

# c) compile & train
vae.compile(optimizer=tf.keras.optimizers.Adam(1e-3))
# assume `train_dataset` yields batches of shape (batch,64,64,64,1)
# vae.fit(train_dataset, epochs=50, batch_size=32)

fn = 'data/test'

In [37]:
encoder.summary(line_length=100)

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                ┃ Output Shape            ┃        Param # ┃ Connected to            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_input (InputLayer)  │ (None, 64, 64, 64, 1)   │              0 │ -                       │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ conv3d_20 (Conv3D)          │ (None, 32, 32, 32, 64)  │          1,792 │ encoder_input[0][0]     │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ conv3d_21 (Conv3D)          │ (None, 16, 16, 16, 128) │        221,312 │ conv3d_20[0][0]         │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ flatten_10 (Flatten)        │ (None, 524288)          │              0 │ conv3d_21[0][0]         │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ dense_30 (Dense)            │ (None, 128)             │     67,108,992 │ flatten_10[0][0]        │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ z_mean (Dense)              │ (None, 32)              │          4,128 │ dense_30[0][0]          │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ z_log_var (Dense)           │ (None, 32)              │          4,128 │ dense_30[0][0]          │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ z (Lambda)                  │ (None, 32)              │              0 │ z_mean[0][0],           │
│                             │                         │                │ z_log_var[0][0]         │
└─────────────────────────────┴─────────────────────────┴────────────────┴─────────────────────────┘

 Total params: 67,340,352 (256.88 MB)

 Trainable params: 67,340,352 (256.88 MB)

 Non-trainable params: 0 (0.00 B)

In [38]:
decoder.summary(line_length=100)

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                               ┃ Output Shape                    ┃           Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ z_sampling (InputLayer)                    │ (None, 32)                      │                 0 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ dense_31 (Dense)                           │ (None, 128)                     │             4,224 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ dense_32 (Dense)                           │ (None, 524288)                  │        67,633,152 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ reshape_10 (Reshape)                       │ (None, 16, 16, 16, 128)         │                 0 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ conv3d_transpose_20 (Conv3DTranspose)      │ (None, 32, 32, 32, 128)         │           442,496 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ conv3d_transpose_21 (Conv3DTranspose)      │ (None, 64, 64, 64, 64)          │           221,248 │
├────────────────────────────────────────────┼─────────────────────────────────┼───────────────────┤
│ decoder_output (Conv3DTranspose)           │ (None, 64, 64, 64, 1)           │             1,729 │
└────────────────────────────────────────────┴─────────────────────────────────┴───────────────────┘

 Total params: 68,302,849 (260.55 MB)

 Trainable params: 68,302,849 (260.55 MB)

 Non-trainable params: 0 (0.00 B)

In [39]:
from tqdm import tqdm
import numpy as np
import pickle

losses = []
nbatches = 100
batch_size = 32

# 1) Make sure data has channel dim
BRAIN_data = BRAIN_data[..., np.newaxis]

for i in tqdm(range(nbatches)):
    # 2) Sample a random batch
    idx = np.random.randint(0, BRAIN_data.shape[0], size=batch_size)
    x_batch = BRAIN_data[idx]

    # 3) Train on batch, unpack history
    h = vae.train_on_batch(x_batch)
    total_loss, recon_loss, kl_loss = h[:3]
    losses.append(total_loss)

    # 4) Compute a quick MSE on the first example
    x_rec = vae.predict(x_batch[:1])
    mse = np.mean((x_batch[0] - x_rec[0])**2)

    # 5) Every 5 iterations, visualize one slice
    if i % 5 == 0:
        slice_idx = x_batch.shape[1] // 2
        im_gt = x_batch[0, slice_idx, ..., 0]
        im_rec = x_rec[0, slice_idx, ..., 0]
        plot_trainProgress(losses, im_rec, im_gt)

    # 6) Every 10 iterations, checkpoint
    if i % 10 == 0:
        pickle.dump(losses, open(fn + '_loss.pickle','wb'))
        vae.save_weights(fn)

    # 7) Early stop on MSE
    if mse < 5e-4:
        print(f"Stopping early at batch {i}, mse={mse:.6f}")
        break

  0%|                                                                          | 0/100 [00:00<?, ?it/s]


ValueError: None values not supported.

# train CVAE

In [11]:
train_cvae = True

In [ ]:
if train_cvae:
    
    latent_dim = 16
    batch_size = 16
    beta = 1
    gamma = 100
    disentangle = True
    cvae, z_encoder, s_encoder, cvae_decoder = get_MRI_CVAE_3D(latent_dim=latent_dim,beta=beta, disentangle=disentangle, gamma=gamma, bias=True, batch_size = batch_size)
    loss = list()    
    
    fdir = 'tf_outputs/CVAE/'
    fn = 'CVAE_weights'
    
    fn = os.path.join(fdir,fn)
    loss = []

ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.ops`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [ ]:
print(fn)

In [26]:
encoder.summary(line_length=100)

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                ┃ Output Shape            ┃        Param # ┃ Connected to            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_input (InputLayer)  │ (None, 64, 64, 64, 1)   │              0 │ -                       │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ conv3d_14 (Conv3D)          │ (None, 32, 32, 32, 64)  │          1,792 │ encoder_input[0][0]     │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ conv3d_15 (Conv3D)          │ (None, 16, 16, 16, 128) │        221,312 │ conv3d_14[0][0]         │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ flatten_7 (Flatten)         │ (None, 524288)          │              0 │ conv3d_15[0][0]         │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ dense_21 (Dense)            │ (None, 128)             │     67,108,992 │ flatten_7[0][0]         │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ z_mean (Dense)              │ (None, 32)              │          4,128 │ dense_21[0][0]          │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ z_log_var (Dense)           │ (None, 32)              │          4,128 │ dense_21[0][0]          │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ z (Lambda)                  │ (None, 32)              │              0 │ z_mean[0][0],           │
│                             │                         │                │ z_log_var[0][0]         │
└─────────────────────────────┴─────────────────────────┴────────────────┴─────────────────────────┘

 Total params: 67,340,352 (256.88 MB)

 Trainable params: 67,340,352 (256.88 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
cvae_decoder.summary(line_length=100)

In [ ]:
DX_batch = DX_subs[np.random.randint(low=0,high=DX_subs.shape[0],size=batch_size),:,:,:];
TD_batch = TD_subs[np.random.randint(low=0,high=TD_subs.shape[0],size=batch_size),:,:,:];

if len(loss)==0:
    loss.append(np.nan)
    im,im1,ss = cvae_query(BRAIN_data,s_encoder,z_encoder,cvae_decoder);
    plot_trainProgress(loss,im,im1);
    loss = list()
else:
    im,im1,ss = cvae_query(BRAIN_data,s_encoder,z_encoder,cvae_decoder);
    plot_trainProgress(loss,im,im1);

In [ ]:
nbatches = 5000
if train_cvae:
    for i in tqdm(range(1,nbatches)):
        
        DX_batch = DX_subs[np.random.randint(low=0,high=DX_subs.shape[0],size=batch_size),:,:,:];
        TD_batch = TD_subs[np.random.randint(low=0,high=TD_subs.shape[0],size=batch_size),:,:,:];
        
        hist = cvae.train_on_batch([DX_batch,TD_batch]);
        loss.append(hist);
        
        mse = ((np.array([DX_batch,TD_batch])-np.array(cvae.predict([DX_batch,TD_batch]))[:,:,:,:,:,0])**2).mean()

        assert not np.isnan(hist),'loss is NaN - somethings wrong'

        im,im1,ss = cvae_query(BRAIN_data, s_encoder, z_encoder, cvae_decoder); 

        

        if np.mod(i,5)==0: # Plot training progress
            plot_trainProgress(loss,im,im1);
            pickle.dump(loss,open(fn+'_loss.pickle','wb'))
            plot_four(DX_batch, TD_batch, z_encoder, s_encoder,cvae_decoder,cvae,idx=0)
            plot_four(DX_batch, TD_batch, z_encoder, s_encoder,cvae_decoder,cvae,idx=1)

        if np.mod(i,10)==0: # Save every 10 batches
            cvae.save_weights(fn)
            
        if mse < .0005:
            break

In [ ]:
import pickle
import os

path = os.path.expanduser('~/Documents/Bilingualism_CVAE/data/tf_outputs/CVAE/CVAE_weights_loss.pickle')
with open(path, 'rb') as handle:
    loaded_object = pickle.load(handle)

In [ ]:
len(loaded_object)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(1000,len(loaded_object)), loaded_object[1000:])
plt.show()

In [ ]:
# !module list
# !nvidia-smi
# import os
# os.environ['CUDA_VISIBLE_DEVICES']='0'
# from datetime import datetime;now = datetime.now;t00 = now()
# print(t00)
# # GPU checks
# import tensorflow as tf
# # tf.test.gpu_device_name()
# # tf.test.is_gpu_available()
# tf.test.gpu_device_name()
# import timeit,pickle
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   print(
#       '\n\nThis error most likely means that this notebook is not '
#       'configured to use a GPU.  Change this in Notebook Settings via the '
#       'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
#   raise SystemError('GPU device not found')

# def cpu():
#   with tf.device('/cpu:0'):
#     random_image_cpu = tf.random.normal((100, 100, 100, 3))
#     net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
#     return tf.math.reduce_sum(net_cpu)

# def gpu():
#   with tf.device('/device:GPU:0'):
#     random_image_gpu = tf.random.normal((100, 100, 100, 3))
#     net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
#     return tf.math.reduce_sum(net_gpu)
  
# # We run each op once to warm up; see: https://stackoverflow.com/a/45067900
# cpu()
# gpu()

# # Run the op several times.
# print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
#       '(batch x height x width x channel). Sum of ten runs.')
# print('CPU (s):')
# cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
# print(cpu_time)
# print('GPU (s):')
# gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
# print(gpu_time)
# print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))